In [ ]:
import pandas as pd
import random
import math

# ----------------------------------------
# Step 1: Load All Input Sheets from BR-2.xlsx
# ----------------------------------------
excel_path = '/content/AA-R1.xlsx'  # adjust if needed

# Load all sheets once
xls = pd.ExcelFile(excel_path)

# 1.1 Renovation Blocks sheet
renovation_block_data = xls.parse('RenovationProgram Table Input 1')
renovation_block_data.columns = renovation_block_data.columns.str.strip()

# Rename columns to match expected names used later in the script
renovation_block_data = renovation_block_data.rename(columns={
    'Cumulative_Block_Circulation_Area': 'Cumulative_Area_SQM',
    'Max_Occupancy_with_Capacity': 'Max_Occupancy_with_Capacity' # This column name seems correct already
})




# Ensure relevant columns are numeric (using the potentially renamed columns)
for df in [renovation_block_data, existing_block_data]:
    df['Cumulative_Area_SQM'] = pd.to_numeric(
        df['Cumulative_Area_SQM'], errors='coerce' # Use coerce to handle potential non-numeric values
    ).fillna(0) # Fill NaN with 0 after coercion
    df['Max_Occupancy_with_Capacity'] = pd.to_numeric(
        df['Max_Occupancy_with_Capacity'], errors='coerce' # Use coerce
    ).fillna(0) # Fill NaN with 0

# Combine renovation and existing blocks for a complete set of blocks to be placed or considered
all_block_data = pd.concat([renovation_block_data], ignore_index=True)


# 1.3 Floors sheet
all_floor_data = xls.parse('Program Table Input 2 - Floor', skiprows=0) # Don't skip header
all_floor_data.columns = all_floor_data.columns.str.strip()
print(all_floor_data.columns.tolist())

all_floor_data = all_floor_data.rename(columns={
    all_floor_data.columns[0]: 'Name',
    all_floor_data.columns[1]: 'Usable_Area_(SQM)',
    all_floor_data.columns[2]: 'Max_Assignable_Floor_loading_Capacity'
})
print(all_floor_data.columns.tolist())
# Coerce floor‐area and capacity to numeric
all_floor_data['Usable_Area_(SQM)'] = pd.to_numeric(
    all_floor_data['Usable_Area_(SQM)'], errors='coerce' # Use coerce
).fillna(0) # Fill NaN with 0
all_floor_data['Max_Assignable_Floor_loading_Capacity'] = pd.to_numeric(
    all_floor_data['Max_Assignable_Floor_loading_Capacity'], errors='coerce' # Use coerce
).fillna(0) # Fill NaN with 0


# 1.4 Department Split sheet (skip header row)
department_split_data = xls.parse('Department Split', header=1) # Set the second row as the header
department_split_data.columns = department_split_data.columns.str.strip()
print(department_split_data.columns.tolist())

# Correctly rename the columns based on the current output
department_split_data = department_split_data.rename(
    columns={
        department_split_data.columns[0]: 'Department_Sub-Department', # The third column is the correct one
        department_split_data.columns[1]: 'Splittable', # Assuming the fourth column is Splittable
        department_split_data.columns[2]: 'Min_%of_Block_per_department' # Assuming the fifth column is Min%_of_Block_per_department
    }
)

print(department_split_data.columns.tolist())
# Build dictionaries:
dept_splittable = department_split_data.set_index('Department_Sub-Department')['Splittable'].to_dict()
dept_min_pct    = department_split_data.set_index('Department_Sub-Department')['Min_%of_Block_per_department'].to_dict()

# 1.5 Min%Split sheet (not used below but loaded)
min_split_data = xls.parse('Min % Split')
min_split_data.columns = min_split_data.columns.str.strip()

# 1.6 Adjacency sheet
adjacency_sheet_name = [name for name in xls.sheet_names if "Adjacency" in name][0]
raw_adj = xls.parse(adjacency_sheet_name, header=1, index_col=0)
adjacency_data = raw_adj.apply(pd.to_numeric, errors='coerce')
adjacency_data.index = adjacency_data.index.str.strip()
adjacency_data.columns = adjacency_data.columns.str.strip()
adj_lookup = adjacency_data.to_dict()

# 1.7 De-Centralized Logic sheet
df_logic = xls.parse('De-Centralized Logic', header=None)
De_Centralized_data = {}
current_section = None
for _, row in df_logic.iterrows():
    first_cell = str(row[0]).strip() if pd.notna(row[0]) else ''
    if first_cell in ['Centralised', 'Semi Centralized', 'DeCentralised']:
        current_section = first_cell
        De_Centralized_data[current_section] = {'Add': 0}
    elif current_section and 'Add' in str(first_cell):
        De_Centralized_data[current_section]['Add'] = int(row[1]) if pd.notna(row[1]) else 0
for key in ['Centralised', 'Semi Centralized', 'DeCentralised']:
    if key not in De_Centralized_data:
        De_Centralized_data[key] = {'Add': 0}

# ----------------------------------------
# Step 2: Preprocess Blocks & Department Split
# ----------------------------------------

# 2.1 Separate Immovable vs. Movable blocks
immovable_blocks = all_block_data[
    all_block_data['Immovable-Movable Asset'] == 'Immovable Asset'
].copy()
movable_blocks = all_block_data[
    all_block_data['Immovable-Movable Asset'] == 'Movable Asset'
].copy()

# 2.2 Separate Destination vs. Typical blocks from movable blocks
destination_blocks = movable_blocks[
    movable_blocks['Typical_Destination'] == 'Destination'
].copy()
typical_blocks = movable_blocks[
    movable_blocks['Typical_Destination'] == 'Typical'
].copy()

# ----------------------------------------
# Step 3: Initialize Floor Assignments
# ----------------------------------------

def initialize_floor_assignments(floor_df):
    """
    Returns a dict keyed by floor name. Each entry has:
      - remaining_area
      - remaining_capacity
      - assigned_blocks      (list of block‐row dicts)
      - assigned_departments (set of sub‐departments)
      - ME_area, WE_area, US_area, Support_area, Speciality_area (floats)
    """
    assignments = {}
    for _, row in floor_df.iterrows():
        floor = row['Name'].strip()
        assignments[floor] = {
            'remaining_area': row['Usable_Area_(SQM)'],
            'remaining_capacity': row['Max_Assignable_Floor_loading_Capacity'],
            'assigned_blocks': [],
            'assigned_departments': set(),
            'DeptArea': {},  # track area per sub-dept
            'ME_area': 0.0,
            'WE_area': 0.0,
            'US_area': 0.0,
            'Support_area': 0.0,
            'Speciality_area': 0.0
        }
    return assignments

floors = list(initialize_floor_assignments(all_floor_data).keys())

# Define all_categories outside the function
all_categories = ['ME', 'WE', 'US', 'Support', 'Speciality']

# Helpers for adjacency & splitting
def can_place_block(blk, fl, assignments, mode):
    dept = blk.get('Department_Sub_Department', '').strip() # Use .get for robustness
    # 1) adjacency hard forbid
    for other in assignments[fl]['assigned_departments']:
        if dept in adj_lookup and other in adj_lookup[dept] and adj_lookup[dept][other] == -1:
            return False
    # 2) destination floor lock (decentralized)
    if mode == 'decentralized' and blk.get('Typical_Destination','')=='Destination': # Use .get for robustness
        max_dest = 2 + De_Centralized_data['DeCentralised']['Add']
        if fl not in floors[:max_dest]:
            return False
    # 3) department split rules
    spl = dept_splittable.get(dept, -1)
    if spl == 1:
        # must stay on one floor
        for f2 in assignments:
            if f2!=fl and dept in assignments[f2]['assigned_departments']:
                return False
    elif spl == 0:
        # waterfall: enforce min pct first
        min_pct = dept_min_pct.get(dept,100)/100.0
        used = assignments[fl]['DeptArea'].get(dept,0.0) + blk.get('Cumulative_Area_SQM', 0) # Use .get for robustness
        floor_area = all_floor_data.loc[all_floor_data['Name']==fl,'Usable_Area_(SQM)'].iloc[0]
        if used/floor_area < min_pct:
            return False
    # -1 or 0.75: no hard block
    return True

def primary_category(blk):
    mix = blk.get('SpaceMix_(ME_WE_US_Support_Speciality)', '') # Use .get for safety
    for cat in ['ME','WE','US']:
        if cat in mix:
            return cat
    return 'Support' # Default to Support if no match

# ----------------------------------------
# Step 4: Core Stacking Function
# ----------------------------------------
def run_stack_plan(mode):
    assignments = initialize_floor_assignments(all_floor_data)
    unassigned_blocks = []

    # Phase 0: Pre-assign immovable blocks with constraint checks
    for _, blk_series in immovable_blocks.iterrows():
        blk = blk_series.to_dict() # Convert row to dictionary for consistent access
        assigned_level = blk.get('Level', '').strip()
        blk_area = blk.get('Cumulative_Area_SQM', 0)
        blk_capacity = blk.get('Max_Occupancy_with_Capacity', 0)

        # Find the correct floor name based on the Level column
        matching_floor = None
        for floor_name in floors:
            if assigned_level.lower() in floor_name.lower():
                matching_floor = floor_name
                break

        if matching_floor and assignments[matching_floor]['remaining_area'] >= blk_area and assignments[matching_floor]['remaining_capacity'] >= blk_capacity:
            assignments[matching_floor]['assigned_blocks'].append(blk)
            assignments[matching_floor]['remaining_area'] -= blk_area
            assignments[matching_floor]['remaining_capacity'] -= blk_capacity
            assignments[matching_floor]['assigned_departments'].add(blk.get('Department_Sub_Department', '').strip())
            cat = blk.get('SpaceMix_(ME_WE_US_Support_Speciality)', '').strip()
            if cat == 'ME':
                assignments[matching_floor]['ME_area'] += blk_area
            elif cat == 'WE':
                assignments[matching_floor]['WE_area'] += blk_area
            elif cat == 'US':
                assignments[matching_floor]['US_area'] += blk_area
            elif cat.lower() == 'support':
                assignments[matching_floor]['Support_area'] += blk_area
            elif cat.lower() == 'speciality':
                assignments[matching_floor]['Speciality_area'] += blk_area
        else:
            # If an immovable block cannot be placed on its specified floor due to constraints
            unassigned_blocks.append(blk)


    # 4.1 Determine how many floors to use for destination blocks
    def destination_floor_count():
        if mode == 'centralized':
            return 2
        elif mode == 'semi':
            return 2 + De_Centralized_data["Semi Centralized"]["Add"]
        elif mode == 'decentralized':
            return 2 + De_Centralized_data["DeCentralised"]["Add"]
        else:
            return 2

    max_dest_floors = min(destination_floor_count(), len(floors))

    # 4.2 Group destination blocks by Destination_Group
    dest_groups = {}
    for _, blk in destination_blocks.iterrows():
        grp = blk['Destination_Group']
        if grp not in dest_groups:
            dest_groups[grp] = {'blocks': [], 'total_area': 0.0, 'total_capacity': 0}
        dest_groups[grp]['blocks'].append(blk.to_dict())
        dest_groups[grp]['total_area'] += blk['Cumulative_Area_SQM']
        dest_groups[grp]['total_capacity'] += blk['Max_Occupancy_with_Capacity']

    # Phase 1: Assign destination groups (try whole‐group first; if that fails, split across floors)
    group_names = list(dest_groups.keys())
    random.shuffle(group_names)
    for grp in group_names:
        info_grp = dest_groups[grp]
        grp_area = info_grp['total_area']
        grp_cap  = info_grp['total_capacity']
        placed_whole = False

        # 4.2.a Attempt to place entire group on any of the first max_dest_floors
        candidate_floors = floors[:max_dest_floors].copy()

        for fl in candidate_floors:
            if (assignments[fl]['remaining_area'] >= grp_area and
                assignments[fl]['remaining_capacity'] >= grp_cap):
                # Entire group fits here—place all blocks
                for blk in info_grp['blocks']:
                    assignments[fl]['assigned_blocks'].append(blk)
                    assignments[fl]['assigned_departments'].add(
                        blk['Department_Sub_Department']
                    )
                assignments[fl]['remaining_area'] -= grp_area
                assignments[fl]['remaining_capacity'] -= grp_cap
                placed_whole = True
                break

        # 4.2.b If not yet placed, try the remaining floors (beyond max_dest_floors)
        if not placed_whole:
            for fl in floors[max_dest_floors:]:
                if (assignments[fl]['remaining_area'] >= grp_area and
                    assignments[fl]['remaining_capacity'] >= grp_cap):
                    for blk in info_grp['blocks']:
                        assignments[fl]['assigned_blocks'].append(blk)
                        assignments[fl]['assigned_departments'].add(
                            blk['Department_Sub_Department'].strip()
                        )
                    assignments[fl]['remaining_area'] -= grp_area
                    assignments[fl]['remaining_capacity'] -= grp_cap
                    placed_whole = True
                    break

        # 4.2.c If still not placed as a whole, split the group block‐by‐block across floors
        if not placed_whole:
            total_remaining_area = sum(assignments[f]['remaining_area'] for f in floors)
            if total_remaining_area >= grp_area:
                # Try placing group by removing the largest blocks one-by-one until remaining can be placed whole
                blocks_sorted = sorted(info_grp['blocks'], key=lambda b: b['Cumulative_Area_SQM'], reverse=True)
                removed_blocks = []
                trial_blocks = blocks_sorted.copy()

                while trial_blocks:
                    trial_area = sum(b['Cumulative_Area_SQM'] for b in trial_blocks)
                    trial_capacity = sum(b['Max_Occupancy_with_Capacity'] for b in trial_blocks)

                    # Try to place this reduced group
                    floor_combination = []
                    temp_assignments = {f: assignments[f].copy() for f in floors}
                    temp_floors_by_space = sorted(floors, key=lambda f: assignments[f]['remaining_area'], reverse=True)

                    temp_success = True
                    for blk in trial_blocks:
                        blk_area = blk['Cumulative_Area_SQM']
                        blk_capacity = blk['Max_Occupancy_with_Capacity']
                        placed_block = False

                        for fl in temp_floors_by_space:
                            if (temp_assignments[fl]['remaining_area'] >= blk_area and
                                temp_assignments[fl]['remaining_capacity'] >= blk_capacity and
                                can_place_block(blk,fl,temp_assignments,mode)): # Added can_place_block check
                                temp_assignments[fl]['remaining_area'] -= blk_area
                                temp_assignments[fl]['remaining_capacity'] -= blk_capacity
                                floor_combination.append((blk, fl))
                                placed_block = True
                                break

                        if not placed_block:
                            temp_success = False
                            break

                    if temp_success:
                        # Apply final assignment for successfully placed trial blocks
                        for blk, fl in floor_combination:
                            assignments[fl]['assigned_blocks'].append(blk)
                            assignments[fl]['assigned_departments'].add(blk['Department_Sub_Department'].strip())
                            assignments[fl]['remaining_area'] -= blk['Cumulative_Area_SQM']
                            assignments[fl]['remaining_capacity'] -= blk['Max_Occupancy_with_Capacity']
                        placed_whole = True
                        break
                    else:
                        # Remove one largest block and retry
                        removed_blocks.append(trial_blocks.pop(0))

                # Place removed blocks one-by-one
                for blk in removed_blocks:
                    blk_area = blk['Cumulative_Area_SQM']
                    blk_capacity = blk['Max_Occupancy_with_Capacity']
                    placed_block = False
                    floors_by_space = sorted(floors, key=lambda f: assignments[f]['remaining_area'], reverse=True)

                    for fl in floors_by_space:
                        if (assignments[fl]['remaining_area'] >= blk_area and
                            assignments[fl]['remaining_capacity'] >= blk_capacity and
                            can_place_block(blk,fl,assignments,mode)): # Added can_place_block check
                            assignments[fl]['assigned_blocks'].append(blk)
                            assignments[fl]['assigned_departments'].add(blk['Department_Sub_Department'].strip())
                            assignments[fl]['remaining_area'] -= blk_area
                            assignments[fl]['remaining_capacity'] -= blk_capacity
                            placed_block = True
                            break

                    if not placed_block:
                        unassigned_blocks.append(blk)
            else:
                # Even splitting won't fit all blocks, place block-by-block
                for blk in sorted(info_grp['blocks'], key=lambda b: b['Cumulative_Area_SQM'], reverse=True):
                    blk_area     = blk['Cumulative_Area_SQM']
                    blk_capacity = blk['Max_Occupancy_with_Capacity']
                    placed_block = False

                    floors_by_space = sorted(floors, key=lambda f: assignments[f]['remaining_area'], reverse=True)
                    for fl in floors_by_space:
                        if (assignments[fl]['remaining_area'] >= blk_area and
                            assignments[fl]['remaining_capacity'] >= blk_capacity and
                            can_place_block(blk,fl,assignments,mode)): # Added can_place_block check
                            assignments[fl]['assigned_blocks'].append(blk)
                            assignments[fl]['assigned_departments'].add(blk['Department_Sub_Department'].strip())
                            assignments[fl]['remaining_area'] -= blk_area
                            assignments[fl]['remaining_capacity'] -= blk_capacity
                            placed_block = True
                            break

                    if not placed_block:
                        unassigned_blocks.append(blk)


    # Phase 2: Handle typical blocks with department‐splittable logic

    # 4.3 Separate typical blocks into:
    #   - dept_unsplittable_groups: {department → [block_dicts]} for Splittable != -1
    #   - splittable_blocks: list of block_dicts for Splittable == -1
    dept_unsplittable_groups = {}
    splittable_blocks = []

    for blk in typical_blocks.to_dict('records'):
        dept = blk.get('Department_Sub_Department', '').strip() # Use .get for safety
        # ← DEFAULT TO -1 (splittable) IF MISSING
        spl = dept_splittable.get(dept, -1)
        if spl == 1:
            # must stay on one floor
            for f2 in assignments:
                if f2!=fl and dept in assignments[f2]['assigned_departments']:
                    return False
        else:
            dept_unsplittable_groups.setdefault(dept, []).append(blk)

    # 4.4 Phase 2A: Assign each unsplittable department's blocks as a group
    for dept, blocks_list in dept_unsplittable_groups.items():
        total_a = sum(b.get('Cumulative_Area_SQM', 0) for b in blocks_list) # Use .get for safety
        total_c = sum(b.get('Max_Occupancy_with_Capacity', 0) for b in blocks_list) # Use .get for safety
        placed=False

        candidate_floors = sorted(
            floors,
            key=lambda f: assignments[f]['remaining_area'],
            reverse=True
        )
        for fl in candidate_floors:
            if (assignments[fl]['remaining_area']>=total_a and
                assignments[fl]['remaining_capacity']>=total_c and
                all(can_place_block(b,fl,assignments,mode) for b in blocks_list)): # Added can_place_block check
                for blk in blocks_list:
                    assignments[fl]['assigned_blocks'].append(blk)
                    assignments[fl]['assigned_departments'].add(dept)
                    cat=primary_category(blk)
                    assignments[fl][cat+'_area']+=blk.get('Cumulative_Area_SQM', 0) # Use .get for safety
                    assignments[fl]['DeptArea'][dept]=assignments[fl]['DeptArea'].get(dept,0)+blk.get('Cumulative_Area_SQM', 0) # Use .get for safety
                assignments[fl]['remaining_area']-=total_a
                assignments[fl]['remaining_capacity']-=total_c
                placed=True
                break

        if not placed:
            # Mark entire department group as unassigned
            unassigned_blocks.extend(blocks_list)

    # 4.5 Phase 2B: On the remaining splittable blocks, assign by space‐mix logic

    # 4.5.a Assign all ME blocks randomly
    me_blks = [
        blk for blk in splittable_blocks
        if blk.get('SpaceMix_(ME_WE_US_Support_Speciality)', '').strip() == 'ME'
    ]
    random.shuffle(me_blks)
    for blk in me_blks:
        blk_area     = blk.get('Cumulative_Area_SQM', 0) # Use .get for safety
        blk_capacity = blk.get('Max_Occupancy_with_Capacity', 0) # Use .get for safety
        blk_dept     = blk.get('Department_Sub-Department', '').strip() # Use .get for safety


        candidate_floors = floors.copy()
        random.shuffle(candidate_floors)
        placed = False
        for fl in candidate_floors:
            if assignments[fl]['remaining_area'] >= blk_area:
                assignments[fl]['assigned_blocks'].append(blk)
                assignments[fl]['remaining_area'] -= bll_area
                assignments[fl]['remaining_capacity'] -= blk_capacity
                assignments[fl]['assigned_departments'].add(blk_dept)
                assignments[fl]['ME_area'] += blk_area
                placed = True
                break
        if not placed:
            unassigned_blocks.append(blk)

    # compute target distribution
    me_counts = {fl: sum(1 for blk in assignments[fl]['assigned_blocks'] if primary_category(blk)=='ME') for fl in floors} # Use primary_category helper
    tot_me = sum(me_counts.values()) or 1
    frac_me = {f:me_counts[f]/tot_me for f in floors}

    for cat in ['WE','US','Support','Speciality']:
        cat_blks = [b for b in splittable_blocks if primary_category(b)==cat]
        total_cat = len(cat_blks)
        if total_cat==0: continue
        raw_t = {f:frac_me[f]*total_cat for f in floors}
        tgt = {f:int(round(raw_t[f])) for f in floors}
        diff = total_cat - sum(tgt.values())
        if diff>0:
            frac_parts = {f:raw_t[f]-math.floor(raw_t[f]) for f in floors}
            for f in sorted(floors, key=lambda x:frac_parts[x], reverse=True)[:diff]: tgt[f]+=1
        elif diff<0:
            frac_parts = {f:raw_t[f]-math.floor(raw_t[f]) for f in floors}
            for f in sorted(floors, key=lambda x:frac_parts[x])[: -diff]: tgt[f]-=1
        random.shuffle(cat_blks)
        assigned={f:0 for f in floors}
        for blk in cat_blks:
            placed=False
            blk_area = blk.get('Cumulative_Area_SQM', 0) # Use .get for safety
            blk_capacity = blk.get('Max_Occupancy_with_Capacity', 0) # Use .get for safety
            blk_dept = blk.get('Department_Sub-Department', '').strip() # Use .get for safety

            deficits={f:tgt[f]-assigned[f] for f in floors}
            cand = [f for f in floors if deficits[f]>0] or floors
            for fl in sorted(cand, key=lambda x:deficits.get(x,0), reverse=True):
                if (assignments[fl]['remaining_area']>=blk_area and
                    assignments[fl]['remaining_capacity']>=blk_capacity and
                    can_place_block(blk,fl,assignments,mode)): # Added can_place_block check
                    assignments[fl]['assigned_blocks'].append(blk)
                    assignments[fl]['remaining_area']    -= blk_area
                    assignments[fl]['remaining_capacity']-= blk_capacity
                    assignments[fl][cat+'_area']+=blk_area
                    assigned[fl]+=1
                    placed=True
                    break
            if not placed:
                unassigned_blocks.append(blk)

    # Phase 3: Attempt to assign remaining unassigned blocks
    still_unassigned = []
    for blk in unassigned_blocks:
        blk_area = blk.get('Cumulative_Area_SQM', 0)
        blk_capacity = blk.get('Max_Occupancy_with_Capacity', 0)
        placed = False
        # Try to place on any floor, prioritizing by remaining area
        for fl in sorted(floors, key=lambda x: assignments[x]['remaining_area'], reverse=True):
             if (assignments[fl]['remaining_area'] >= blk_area and
                assignments[fl]['remaining_capacity'] >= blk_capacity and
                can_place_block(blk, fl, assignments, mode)):
                assignments[fl]['assigned_blocks'].append(blk)
                assignments[fl]['remaining_area'] -= blk_area
                assignments[fl]['remaining_capacity'] -= blk_capacity
                assignments[fl]['assigned_departments'].add(blk.get('Department_Sub_Department', '').strip())
                cat = blk.get('SpaceMix_(ME_WE_US_Support_Speciality)', '').strip()
                if cat == 'ME':
                    assignments[fl]['ME_area'] += blk_area
                elif cat == 'WE':
                    assignments[fl]['WE_area'] += blk_area
                elif cat == 'US':
                    assignments[fl]['US_area'] += blk_area
                elif cat.lower() == 'support':
                    assignments[fl]['Support_area'] += blk_area
                elif cat.lower() == 'speciality':
                    assignments[fl]['Speciality_area'] += blk_area
                placed = True
                break
        if not placed:
            still_unassigned.append(blk)

    unassigned_blocks = still_unassigned # Update the list of unassigned blocks


    # Build output DataFrames
    rows=[]
    for fl,info in assignments.items():
        for blk in info['assigned_blocks']:
            rows.append({
                'Floor':fl,
                'Department':blk.get('Department_Sub_Department', ''), # Use .get for robustness
                'Block_Name':blk.get('Block_Name', ''), # Use .get for robustness
                'Destination_Group':blk.get('Destination_Group', ''), # Use .get for robustness
                'SpaceMix': blk.get('SpaceMix_(ME_WE_US_Support_Speciality)', ''), # Use .get for robustness
                'Assigned_Area_SQM':blk.get('Cumulative_Area_SQM', 0), # Use .get for robustness
                'Max_Occupancy':blk.get('Max_Occupancy_with_Capacity', 0), # Use .get for robustness
                'Immovable-Movable Asset': blk.get('Immovable-Movable Asset', '') # Add this column
            })
    detailed_df = pd.DataFrame(rows) if rows else pd.DataFrame(columns=['Floor', 'Department', 'Block_Name', 'Destination_Group', 'SpaceMix', 'Assigned_Area_SQM', 'Max_Occupancy', 'Immovable-Movable Asset'])

    summary = (
        detailed_df.groupby('Floor')
        .agg(Assgn_Blocks=('Block_Name','count'),
             Assgn_Area_SQM=('Assigned_Area_SQM','sum'),
             Total_Occupancy=('Max_Occupancy','sum'))
        .reset_index()
    ) if not detailed_df.empty else pd.DataFrame(columns=['Floor', 'Assgn_Blocks', 'Assgn_Area_SQM', 'Total_Occupancy'])

    input_sub = all_floor_data[['Name','Usable_Area_(SQM)','Max_Assignable_Floor_loading_Capacity']].rename(columns={'Name':'Floor','Usable_Area_(SQM)':'Input_Usable_Area_SQM','Max_Assignable_Floor_loading_Capacity':'Input_Max_Capacity'})
    floor_summary_df = pd.merge(input_sub, summary, on='Floor', how='left').fillna(0)

    # space-mix by %
    # Recalculate category totals based on the original all_block_data (renovation + existing)
    category_totals = {
        cat: len(all_block_data[
            all_block_data['SpaceMix_(ME_WE_US_Support_Speciality)'].astype(str).str.strip() == cat
        ])
        for cat in all_categories
    }

    mix_rows = [] # Renamed from rows to mix_rows to avoid confusion
    for fl, info in assignments.items():
        counts = {cat: 0 for cat in all_categories}
        for blk in info['assigned_blocks']:
            cat = blk.get('SpaceMix_(ME_WE_US_Support_Speciality)', '').strip() # Use .get and strip for safety
            if cat in counts:
                counts[cat] += 1
        total_blocks_on_floor = sum(counts.values())

        for cat in all_categories:
            cnt = counts[cat]
            # Percent of floor’s blocks
            pct_of_floor = (cnt / total_blocks_on_floor * 100) if total_blocks_on_floor else 0.0
            # Percent of overall blocks of that category
            total_cat = category_totals.get(cat, 0) # Use .get with default 0
            pct_overall = (cnt / total_cat * 100) if total_cat else 0.0

            mix_rows.append({ # Append to mix_rows
                'Floor': fl,
                'SpaceMix': cat,
                '%spaceMix': round(pct_overall, 2)

            })

    space_mix_df = pd.DataFrame(mix_rows) if mix_rows else pd.DataFrame(columns=['Floor', 'SpaceMix', '%spaceMix'])


    # 4.6.4 Unassigned DataFrame
    unassigned_list = []
    for blk in unassigned_blocks:
        unassigned_list.append({
            'Department': blk.get('Department_Sub_Department', ''),
            'Block_Name': blk.get('Block_Name', ''),
            'Destination_Group': blk.get('Destination_Group', ''),
            'SpaceMix': blk.get('SpaceMix_(ME_WE_US_Support_Speciality)', ''),
            'Area_SQM': blk.get('Cumulative_Area_SQM', 0),
            'Max_Occupancy': blk.get('Max_Occupancy_with_Capacity', 0),
            'Immovable-Movable Asset': blk.get('Immovable-Movable Asset', '') # Add this column
        })
    unassigned_df = pd.DataFrame(unassigned_list) if unassigned_list else pd.DataFrame(columns=['Department', 'Block_Name', 'Destination_Group', 'SpaceMix', 'Area_SQM', 'Max_Occupancy', 'Immovable-Movable Asset'])


    return detailed_df, floor_summary_df, space_mix_df, unassigned_df

# ----------------------------------------
# Step 5: Generate & Export Excel + CSV Files (including Unassigned)
# ----------------------------------------

central_detailed, central_floor_sum, central_space_mix, central_unassigned = run_stack_plan('centralized')
semi_detailed,    semi_floor_sum,    semi_space_mix,    semi_unassigned    = run_stack_plan('semi')
decentral_detailed, decentral_floor_sum, decentral_space_mix, decentral_unassigned = run_stack_plan('decentralized')

# File names
central_file    = 'AAR1_stack_plan_centralized14.xlsx'
semi_file       = 'AAR1_stack_plan_semi_centralized14.xlsx'
decentral_file  = 'AAR1_stack_plan_decentralized14.xlsx'

# --- ExcelWriter blocks with an extra sheet "Unassigned" ---
with pd.ExcelWriter(central_file) as writer:
    central_detailed.to_excel(writer, sheet_name='Detailed', index=False)
    central_floor_sum.to_excel(writer, sheet_name='Floor_Summary', index=False)
    central_space_mix.to_excel(writer, sheet_name='SpaceMix_By_Units', index=False)
    central_unassigned.to_excel(writer, sheet_name='Unassigned', index=False)

with pd.ExcelWriter(semi_file) as writer:
    semi_detailed.to_excel(writer, sheet_name='Detailed', index=False)
    semi_floor_sum.to_excel(writer, sheet_name='Floor_Summary', index=False)
    semi_space_mix.to_excel(writer, sheet_name='SpaceMix_By_Units', index=False)
    semi_unassigned.to_excel(writer, sheet_name='Unassigned', index=False)

with pd.ExcelWriter(decentral_file) as writer:
    decentral_detailed.to_excel(writer, sheet_name='Detailed', index=False)
    decentral_floor_sum.to_excel(writer, sheet_name='Floor_Summary', index=False)
    decentral_space_mix.to_excel(writer, sheet_name='SpaceMix_By_Units', index=False)
    decentral_unassigned.to_excel(writer, sheet_name='Unassigned', index=False)

print("✅ Generated three Excel outputs (each with an 'Unassigned' sheet):")
print(f"    • {central_file}")
print(f"    • {semi_file}")
print(f"    • {decentral_file}")